# Exploring Venues for new Shopping Mall in New Delhi, India

## Table of Contents

* [Introduction](#introduction)
* [Data Collection](#data_collection)

## Introduction <a name="introduction"></a>

The scope of the project is to identify venues to open a new shopping mall in New Delhi, India. 

In this notebook, we will identify various venues in the city of **New Delhi, India** to help some who wants to open a new shopping mall across all major venues of the capital city of India. Data has been scrapped using **Wikipedia** and **Foursquare API**.

After analysing the venues data, we will cluster the data and select the best one or two cluster to open a new venture.

## Data Collection <a name="data_collection"></a>

Intially, we will get the list of major venues in New Delhi. For this we will exract the list from a wikipedia page where we have list of neighbourhood of the city.

Since we have only extracted the list of venues, we will require the lattitude and longitude corrdinates. We will use Nomatim and Geocoder libraries for this purpose.

Once we have the list of venues and their corrdinates, our next aim will be to locate the shopping malls across the city and for this purpose we will be using the Foursquare API.

In [1]:
#importing Libraries
import requests
import lxml.html as lh
import bs4 as bs
import urllib.request
import numpy as np 
import pandas as pd

#### Getting the list of neighbourhood locations from Wikipedia using Beautiful Library

In [2]:
#Getting the data from url
url = "https://en.wikipedia.org/wiki/List_of_neighbourhoods_of_Delhi"
res = requests.get(url)
soup = bs.BeautifulSoup(res.content,'lxml')
l1=[]
table = soup.find_all('li')
for all in table:
    l1.append(all.text)

#Since our targeted list of values were 'Adarsh Nagar' to 'Viakspuri' so extracting those only.

ncr_df = pd.DataFrame({"Neighborhood": l1[l1.index('Adarsh Nagar'):l1.index('Vikaspuri')+1]})

print(ncr_df.shape)
ncr_df.head()

(118, 1)


,Neighborhood
0,Adarsh Nagar
1,Ashok Vihar
2,Begum Pur
3,Karala
4,Model Town


In [3]:
## Removing 'edit' phrase coming with data
#ncr_df=ncr_df[ncr_df['Neighborhood']!='edit']
#ncr_df.shape

In [4]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

#### Generating the Lattitude & Longitudes Corrdinates of each extracted venue

In [5]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, new Delhi, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [6]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in ncr_df["Neighborhood"].tolist() ]
coords

[[28.720350000000053, 77.17264000000006],
 [28.69037000000003, 77.17609000000004],
 [28.732620000000054, 77.05213000000003],
 [28.737330000000043, 77.03959000000003],
 [28.705010000000073, 77.18950000000007],
 [28.83979000000005, 77.07696000000004],
 [28.69590000000005, 77.13725000000005],
 [28.733560000000068, 77.10401000000007],
 [28.714230000000043, 77.15744000000007],
 [28.67671000000007, 77.21767000000006],
 [28.620430000000056, 77.04941000000008],
 [28.683760000000063, 77.20163000000008],
 [28.663950000000057, 77.23520000000008],
 [28.598039176797986, 77.12820592344048],
 [28.66366000000005, 77.22492000000005],
 [28.59051000000005, 77.12026000000003],
 [28.665830000000028, 77.18481000000008],
 [28.67557000000005, 77.16190000000006],
 [28.666000000000054, 77.21520000000004],
 [28.700780000000066, 77.22116000000005],
 [28.732790000000023, 77.22438000000005],
 [28.679040000000043, 77.31476000000004],
 [28.67369000000008, 77.28326000000004],
 [28.68249000000003, 77.25651000000005],
 

In [7]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

#### Merging the coordinates into the original dataframe

In [8]:
ncr_df['Latitude'] = df_coords['Latitude']
ncr_df['Longitude'] = df_coords['Longitude']

# check the neighborhoods and the coordinates
print(ncr_df.shape)
ncr_df.head(10)

(118, 3)


,Neighborhood,Latitude,Longitude
0,Adarsh Nagar,28.72035,77.17264
1,Ashok Vihar,28.69037,77.17609
2,Begum Pur,28.73262,77.05213
3,Karala,28.73733,77.03959
4,Model Town,28.70501,77.18950
5,Narela,28.83979,77.07696
6,Pitam Pura,28.69590,77.13725
7,Rohini Sub City,28.73356,77.10401
8,Shalimar Bagh,28.71423,77.15744
9,Civil Lines,28.67671,77.21767



#### Visualising the venues of New Delhi on the map

In [9]:
import folium
import json 
import matplotlib.cm as cm
import matplotlib.colors as colors

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("Imported!")

Imported!


In [10]:
# New Delhi map

geolocator = Nominatim(user_agent="coursera")
address = 'New Delhi, India'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

my_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(ncr_df['Latitude'], ncr_df['Longitude'], ncr_df['Neighborhood']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    
my_map

The geograpical coordinates of New Delhi, India are 28.6138954, 77.2090057.


#### Exploring (Top 100) Venues (in 2000m radius) of each Neighbourhood in New Delhi

In [22]:
#Foursquare credentials

CLIENT_ID = 'HIDING_FOR_PRIVACY' # your Foursquare ID
CLIENT_SECRET = 'HIDING_FOR_PRIVACY' # your Foursquare Secret
ACCESS_TOKEN = 'HIDING_FOR_PRIVACY' # your FourSquare Access Token
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: Hidden')
print('CLIENT_SECRET: Hidden')

Your credentails:
CLIENT_ID: Hidden
CLIENT_SECRET: Hidden


In [12]:
#Extracting venues

radius= 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(ncr_df['Latitude'], ncr_df['Longitude'], ncr_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [13]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head(25)

(5069, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adarsh Nagar,28.72035,77.17264,Shalimar bagh club,28.711882,77.157933,Indian Restaurant
1,Adarsh Nagar,28.72035,77.17264,Ever Bake,28.706419,77.162607,Bakery
2,Adarsh Nagar,28.72035,77.17264,Arjun Pao Bhaji,28.708468,77.184963,Indian Restaurant
3,Adarsh Nagar,28.72035,77.17264,Domino's Pizza,28.706000,77.163000,Pizza Place
4,Adarsh Nagar,28.72035,77.17264,Cafe Coffee Day,28.707934,77.161478,Coffee Shop
5,Adarsh Nagar,28.72035,77.17264,Adarsh Nagar Metro Station,28.716598,77.170436,Light Rail Station
6,Adarsh Nagar,28.72035,77.17264,Jahangirpuri Metro Station | जहांगीरपुरी,28.726026,77.162626,Light Rail Station
7,Adarsh Nagar,28.72035,77.17264,Rameshwar Nagar Park,28.711492,77.181494,Park
8,Adarsh Nagar,28.72035,77.17264,Azadpur,28.708669,77.179102,Bus Station
9,Adarsh Nagar,28.72035,77.17264,McDonald's,28.713312,77.157928,Restaurant


In [14]:
venues_df.groupby(['Neighborhood', 'VenueCategory']).count().head()

Latitude  Longitude  VenueName  VenueLatitude  \
Neighborhood VenueCategory                                                      
Adarsh Nagar Bakery                    1          1          1              1   
             Bus Station               1          1          1              1   
             Coffee Shop               1          1          1              1   
             Golf Course               1          1          1              1   
             Indian Restaurant         2          2          2              2   

                                VenueLongitude  
Neighborhood VenueCategory                      
Adarsh Nagar Bakery                          1  
             Bus Station                     1  
             Coffee Shop                     1  
             Golf Course                     1  
             Indian Restaurant               2

In [15]:
venues_df.groupby(['Neighborhood','VenueCategory']).count().tail(10)

Latitude  Longitude  VenueName  \
Neighborhood VenueCategory                                        
Vivek Vihar  Track                      1          1          1   
             Train Station              1          1          1   
Wazirabad    Accessories Store          1          1          1   
             Chinese Restaurant         1          1          1   
Yamuna Vihar ATM                        3          3          3   
             Clothing Store             1          1          1   
             IT Services                1          1          1   
             Ice Cream Shop             1          1          1   
             Park                       1          1          1   
             Pizza Place                1          1          1   

                                 VenueLatitude  VenueLongitude  
Neighborhood VenueCategory                                      
Vivek Vihar  Track                           1               1  
             Train Station                   1               1  
Wazirabad    Accessories Store               1               1  
             Chinese Restaurant              1               1  
Yamuna Vihar ATM                             3               3  
             Clothing Store                  1               1  
             IT Services                     1               1  
             Ice Cream Shop                  1               1  
             Park                            1               1  
             Pizza Place                     1               1

In [16]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 207 uniques categories.


In [17]:
# print out the list of categories
#venues_df['VenueCategory'].unique()
venues_df['VenueCategory'].value_counts()

Indian Restaurant                  549
Café                               368
Coffee Shop                        313
Hotel                              240
Fast Food Restaurant               234
Pizza Place                        214
Restaurant                         167
Market                             150
Chinese Restaurant                 144
Italian Restaurant                 122
Bar                                109
Bakery                              96
Sandwich Place                      93
Asian Restaurant                    89
Donut Shop                          88
Lounge                              85
Park                                78
Dessert Shop                        69
Snack Place                         69
Shopping Mall                       56
Light Rail Station                  54
BBQ Joint                           50
Clothing Store                      41
Ice Cream Shop                      40
Multiplex                           38
Japanese Restaurant      


#### Analyzing each neighborhood

In [18]:
ncr_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ncr_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ncr_onehot.columns[-1]] + list(ncr_onehot.columns[:-1])
ncr_onehot = ncr_onehot[fixed_columns]

print(ncr_onehot.shape)
ncr_onehot.head()

(5069, 208)


,Neighborhoods,ATM,Accessories Store,Airport,Airport Food Court,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Breakfast Spot,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Cafeteria,Café,Campground,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Nightclub,Nightlife Spot,North Indian Restaurant,Northeast Indian Restaurant,Office,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Pharmacy,Photography Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Punjabi Restaurant,Racetrack,Recreation Center,Resort,Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Udupi Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Zoo
0,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [19]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

ncr_grouped = ncr_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(ncr_grouped.shape)
ncr_grouped

(118, 208)


,Neighborhoods,ATM,Accessories Store,Airport,Airport Food Court,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Breakfast Spot,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Cafeteria,Café,Campground,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Nightclub,Nightlife Spot,North Indian Restaurant,Northeast Indian Restaurant,Office,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Pharmacy,Photography Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Punjabi Restaurant,Racetrack,Recreation Center,Resort,Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Udupi Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Zoo
0,Adarsh Nagar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [20]:
len(ncr_grouped[ncr_grouped["Shopping Mall"] > 0])

39

#### Creating a new DataFrame for Shopping Mall data only

In [21]:
ncr_mall = ncr_grouped[["Neighborhoods","Shopping Mall"]]
print(ncr_mall.shape)
ncr_mall.head()

(118, 2)


,Neighborhoods,Shopping Mall
0,Adarsh Nagar,0.000000
1,Alaknanda,0.000000
2,Anand Vihar,0.029412
3,Ashok Nagar,0.000000
4,Ashok Vihar,0.000000
